In [ ]:
# BERT는 현재 시점에서 텍스트 마이닝 딥러닝 모형 중 가장 중요한 모형이다. 텍스트 마이닝의 거의 전 분야에서 우수한 성능을 보이고 있다.
# 언어 모델은 문장 혹은 단어의 시퀀스에 대해 확률을 할당하는 모델을 말한다.
# BERT는 언어 모델 기반의 학습을 이용해 언어의 대한 이해를 높이는 데 목적이 있다. 물론 그 이해를 바탕으로 다양한 작업을 수행하는 것이
# 최종 목적이다.
# GPT가 단방향 혹은 순방향으로의 셀프 어텐션만 사용하는 것에 대해 BERT는 양방향 셀프 어텐션을 모두 활용할 수 있다.
# GPT는 BERT보다 먼저 발표된 모형으로, 주로 자연어 문장 생성에 특화된 모델이다. 언어 모델로 학습이 되었고 이를 바탕으로 해서 생성 외에
# 문서 분류와 같은 다른 분야에서도 좋은 성능을 냈는데, 트랜스포머의 디코더 부분만 따로 떼어서 학습 모형으로 사용했다.
# 따라서 인코더에서 디코더로의 어텐션은 생략돼 있고, 셀프 어텐션은 순방향만 적용된다.
# 반면, BERT는 트랜스포머의 인코더 부분을 떼어서 사용했다.

# BERT의 학습은 사전학습과 미세조정학습의 두 단계로 나뉘어진다.
# 사전학습은 언어에 대한 이해를 높이기 위한 비지도학습이고, 미세조정학습은 실제 수행하고자 하는 작업에 대한 지도학습이다.

In [1]:
import transformers
from transformers import pipeline

clf = pipeline("sentiment-analysis")
result = clf("what a beautiful day!")[0]
print('감성분석 결과: %s, 감성스코어: %0.4f' % (result['label'], result['score']))

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


감성분석 결과: POSITIVE, 감성스코어: 0.9999


In [2]:
# 텍스트 생성, 비교적 매끄럽게 문장이 만들어지는 것을 볼 수 있다.
from transformers import pipeline

text_generator = pipeline("text-generation")
result = text_generator("Alice was beginning to get very tired of sitting by her sister on the bank,")
print(result[0]['generated_text'])

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


C:\Users\Leedonghoon\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Leedonghoon\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
C:\Users\Leedonghoon\anaconda3\Lib\site-packages\transformers\gen

Alice was beginning to get very tired of sitting by her sister on the bank, and felt that something would have to go off if she turned the whole island around and asked the next man if he had any clothes on them.

"Why didn


In [4]:
# 1이 True, 즉 '의미적으로 유사하다' 유사할 확률 71%

# 자동 클래스를 이용한 토크나이저와 모형의 사용
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Auto Classes를 이용해 사전학습된 내용에 맞는 토크나이저와 모형을 자동으로 설정
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased-finetuned-mrpc"
)

# 의미적으로 유사한 두 문장을 선언
input_sentence = "She angered me with her inappropriate comments, rumor-spreading, and disrespectfulness at the formal dinner table"
target_sequence = "She made me angry when she was rude at dinner"
# 토큰화
tokens = tokenizer(input_sentence, target_sequence, return_tensors="pt")

# 모형으로 결과를 예측
logits = model(**tokens).logits

# 소프트맥스를 이용해 결과값을 클래스에 대한 확률로 변환
results = torch.softmax(logits, dim=1).tolist()[0]
for i, label in enumerate(['no', 'yes']):
    print(f'{label}: {int(round(results[i] * 100))}%')

no: 29%
yes: 71%


In [5]:
# 전혀 관련 없는 문장 실행
target_sequence = "The boy quickly ran across the finish line, seizing yet another victory"
tokens = tokenizer(input_sentence, target_sequence, return_tensors="pt")
logits = model(**tokens).logits
results = torch.softmax(logits, dim=1).tolist()[0]

for i, label in enumerate(['no', 'yes']):
    print(f'{label}:{int(round(results[i] * 100 ))}%')

no:95%
yes:5%


In [6]:
import nltk
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split
import numpy as np

nltk.download('movie_reviews')

# movie review data에서 file id를 가져옴
fileids = movie_reviews.fileids()

# file id를 이용해 raw text file을 가져옴
reviews = [movie_reviews.raw(fileid) for fileid in fileids]
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids]

# label을 0,1의 값으로 변환
label_dict = {'pos':1, 'neg':0}
y = np.array([label_dict[c] for c in categories])

X_train, X_test, y_train, y_test = train_test_split(reviews,y,test_size=0.2, random_state=7)
print('Train set count:', len(X_train))
print('Test set count:', len(X_test))

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\Leedonghoon\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


Train set count: 1600
Test set count: 400


In [10]:
# BERT는 많은 메모리를 사용하므로 400개의 테스트 데이터셋을 한꺼번에 처리하는 것은 메모리가 아주 많은 경우가 아니라면 거의 불가능에 가깝다.
# 따라서 10개씩 잘라서 모형을 돌리고 결과를 합쳐서 성능 살펴보기
# 미세조정학습이 된 'distilbert-base-uncased-finetuned-sst-2-english' 모형을 가져와 사용함.
# GPU 가속을 확용하기 위해 모형과 연산할 텐서를 GPU로 옮기고, 연산이 끝난 결과를 CPU로 가져오는 과정을 포함함.
# 정확도는 84.25%로 나이브 베이즈(8장)로 얻은 79.7%보다 높은 값일 뿐만 아니라 CNN을 이용해 얻은 81.2%보다도 좋은 값이다.
# 즉 사전학습된 모형을 그대로 사용하는 것이 아니라 내가 가진 학습 데이터를 사용해서 성능을 더 높이려는 노력이
# 미세조정학습이다.
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# cuda를 이용한 GPU 연산이 가능하면 cuda를 사용하고, 아니면 cpu를 사용
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Auto Classes를 이용해 사전학습된 내용에 맞는 토크나이저와 모형을 자동으로 설정
tokenizer = AutoTokenizer.from_pretrained(
    'distilbert-base-uncased-finetuned-sst-2-english'
)
model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased-finetuned-sst-2-english'
)
# 모델을 gpu로 옮겨서 연산을 준비
model = model.to(device)

batch_size = 10 # 모형으로 한번에 예측할 데이터의 수
y_pred = [] # 전체 예측결과를 저장

num_batch = len(y_test) //batch_size

for i in range(num_batch):
    inputs = tokenizer(
        X_test[i*batch_size:(i+1)*batch_size],
        truncation=True,
        padding=True,
        return_tensors="pt"
    )
    # 토큰화 결과를 GPU로 이동
    inputs = inputs.to(device)
    
    # 모형으로 결과를 예측
    logits = model(**inputs).logits
    
    # 결괏값을 클래스에 대한 확률로 변환
    pred = F.softmax(logits, dim=1)
    
    # 예측결과를 CPU로 가져와서 넘파이로 변환한 후,
    # argmax로 확률이 가장 큰 클래스를 선택함
    results = pred.cpu().detach().numpy().argmax(axis=1)
    
    # 전체 예측결과에 추가
    y_pred.extend(results.tolist())
    
#  gpu 메모리를 비움
torch.cuda.empty_cache()

score = sum(y_test == np.array(y_pred))/len(y_test)
print("NLTK 영화리뷰 감성분석 정확도:",score)

NLTK 영화리뷰 감성분석 정확도: 0.8425
